<a href="https://colab.research.google.com/github/athanoid/pycovid19/blob/master/covid19_reg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
"""
A Polynomial Regression model with COVID-19 datasets

Data Repository by Johns Hopkins CSSE: https://github.com/CSSEGISandData/COVID-19

@author: thanos
"""
import pandas as pd
import numpy as np

In [29]:
#import data

# fetch from web
confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

# ---DEPRICATED---
#confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv')
#deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv')
#recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv')

# assign to dataframe
df = confirmed
#show 5 first rows
df.head(5)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,4,4,5,7,7,7,11,16,21,22,22,22,24,24,40,40,74,84,94,110,110,120,170,174
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,10,12,23,33,38,42,51,55,59,64,70,76,89,104,123,146,174,186,197,212,223,243
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,3,5,12,12,17,17,19,20,20,20,24,26,37,48,54,60,74,87,90,139,201,230,264,302,367,409,454,511,584,716
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,39,39,53,75,88,113,133,164,188,224,267,308,334,370,376
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,2,3,3,3,4,4,5,7,7,7


In [30]:
# create datasets from DataFrame

# get available dates in mm/dd/yyyy format
dates = list(df)
dates = dates[4:np.size(dates)]

# select country of interest
country = 'Greece'
loc = df.loc[df['Country/Region'] == country]
idx = int(loc.index.values)
#get only numeric data
num = df._get_numeric_data()
# data preparation 
y = list(num.values[idx,2: int(num.size)]) #get list of values
y = np.array(y) #convert into a numpy array
y = y[~np.isnan(y)] #remove nan entries
x = np.array(range(np.size(y)))#to numpy array
x = x[~np.isnan(x)] #remove nan entries
X = x.reshape(-1, 1) #requires reshaping for poly_reg.fit 

# if nans are removed, check if dates list has the same size as y and equalize
if np.abs(np.size(dates)-np.size(y)) > 0:
    diff = np.size(dates)-np.size(y)
    del dates[-diff]
    print('\nRemoved ' + str(diff) + ' value(s)\n')

print('\nLast 10 points of arrays:')
print('x: ' + str(dates[np.size(dates)-10: np.size(dates)]))
print('y: ' + str(y[np.size(y)-10: np.size(y)]))



Last 10 points of arrays:
x: ['3/22/20', '3/23/20', '3/24/20', '3/25/20', '3/26/20', '3/27/20', '3/28/20', '3/29/20', '3/30/20', '3/31/20']
y: [1600. 2060. 2362. 2995. 3544. 4268. 5170. 5962. 6408. 7443.]


In [31]:
# stacked bar chart
import plotly.graph_objects as go

cnf = confirmed.loc[confirmed['Country/Region'] == country]._get_numeric_data().values[0,np.size(dates)+1]
dth = deaths.loc[deaths['Country/Region'] == country]._get_numeric_data().values[0,np.size(dates)+1]
rcv = recovered.loc[recovered['Country/Region'] == country]._get_numeric_data().values[0,np.size(dates)]  

xxs=[country]
fig = go.Figure(go.Bar(x=xxs, y=[cnf], name='Confirmed'))
fig.add_trace(go.Bar(x=xxs, y=[dth], name='Deaths'))
fig.add_trace(go.Bar(x=xxs, y=[rcv], name='Recovered'))

fig.update_layout(title_text=country + ": Stacked bar-plot for all data", barmode='stack', xaxis={'categoryorder':'array', 'categoryarray':['Confirmed', 'Deaths', 'Recovered']})
fig.show()

In [32]:
# Fitting a Polynomial Regression to the dataset
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score

poly_reg = PolynomialFeatures(degree=4)
X_poly = poly_reg.fit_transform(X)
pol_reg = LinearRegression()
pol_reg.fit(X_poly, y)

y_pred = pol_reg.predict(poly_reg.fit_transform(X))#predicted values

#show coefs & scores
_z,ax,bx,cx,dx = pol_reg.coef_
b = pol_reg.intercept_
print('y = {0} * x^4 + {1} * x^3 + {2} * x^2 + {3} * x + {4}'.format(ax,bx,cx,dx,b))

rsqrd = r2_score(y, y_pred) #r-squared
print('\nr-squared: '+str(rsqrd))

rmse = np.linalg.norm(y_pred - y) / np.sqrt(len(y_pred)) #Root mean squared error 
print('rmse: '+str(rmse))

y = -143.84618200447434 * x^4 + 12.671289908043196 * x^3 + -0.37440033466935896 * x^2 + 0.0035053428171920586 * x + 334.7924821019118

r-squared: 0.9891998983782803
rmse: 168.18432796630307


In [33]:
#plot time series with range-slider
import plotly.graph_objects as go
import plotly.express as px

fig = go.Figure()
fig.add_trace(go.Scatter(
                x=np.array(dates),
                y=y,
                name="Confirmed",
                line_color='deepskyblue',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=np.array(dates),
                y=y_pred,
                name="Predicted",
                line_color='dimgray',
                opacity=0.8))

fig.update_layout(title_text=country + ": Confirmed vs predicted cases",
                  xaxis_rangeslider_visible=True)

fig.show()

In [0]:
# helper functions

def numLen(num):
    '''return number digit length'''
    return len(str(int(abs(num))))


def double_days(x,y,xf,yf):  
    '''
    Approximate when incidents will double.
        Input: x vector, y vector, future x, future y
        Output: days to double incidents (int)
    '''    
    cur = int(y[np.size(y)-1])
    tols = {2:10, 3:100, 4:1000, 5:10000, 6:100000, 7:1000000} # digits of tolerance
    atol = tols[numLen(cur)-1]  #The absolute tolerance parameter 
    print(atol)
    for i in range(0, int(xf[np.size(xf)-1])):
        if np.isclose(np.around(cur*2), np.round(yf[i-1],0), atol=atol):
            dday = i
            break
    return (dday-1) - int(x[np.size(x)-1])

In [35]:
#plot forecasted values
days = 14 #days ahead
x_plus = np.arange(np.size(x),np.size(y)+ days) 
x_plus = np.append([x], [x_plus])#append days to forecast
xf = x_plus.reshape(-1, 1)
yf = pol_reg.predict(poly_reg.fit_transform(xf))

# Visualize the Polymonial Regression results
fig = go.Figure()
fig.add_trace(go.Scatter(
                x=x_plus,
                y=y,
                name="Confirmed",
                line_color='deepskyblue',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=x_plus,
                y=yf,
                name="Predicted with projection",
                line_color='dimgray',
                opacity=0.8))

fig.add_annotation(
            x=len(x)-1,
            y=y[len(y)-1],
            text="Today")
fig.add_annotation(
            x=len(x)+double_days(x,y,xf,yf)-1,
            y=yf[len(y)+double_days(x,y,xf,yf)-1],
            text="Doubled")
fig.add_annotation(
            x=len(x_plus)-1,
            y=yf[len(y)+days-1],
            text=str(days)+" days")
fig.update_annotations(dict(
            xref="x",
            yref="y",
            showarrow=True,
            arrowhead=7,
            ax=0,
            ay=-40
))

fig.update_layout(title_text= country + ": "+ str(days) +" days projection",
                  xaxis_rangeslider_visible=True                
                  )

fig.show()

1000
1000


In [36]:
# plot speed
vel = np.diff(y)
acc = np.diff(vel)

# now the colors
clrred = 'rgb(222,0,0)'
clrgrn = 'rgb(0,222,0)'
clrs  = [clrred if int(vel[i-1]) <= int(vel[i]) else clrgrn for i in range(len(vel))]

fig4 = go.Figure()
# first trace (layer) 
fig4.add_trace(go.Bar(
    x=np.array(dates)[1:len(dates)], 
    y=vel[0:len(vel)],
    name='Speed sign',
    marker=dict(color=clrs)
))
# second trace 
fig4.add_trace(go.Scatter( 
    x=np.array(dates)[1:len(dates)],
    y=vel[0:len(vel)],
    name='Speed evolution',
    line=dict(
        color=('rgb(0,0,222)'),
        width=2,
        dash='dot')
))

fig4.update_layout(title_text= country + ": Speed of confirmed cases",
                  xaxis_rangeslider_visible=True)
fig4.show()

In [37]:
#histogram
vel = np.diff(y)
fig5 = go.Figure()
fig5 = go.Figure(data=[go.Histogram(x=vel)])
fig5.update_layout(title_text= country + ": Distribution of daily cases",
                   xaxis_title_text='Daily cases',
                  yaxis_title_text='Count')
fig5.show()

In [38]:
# print report
cur = [y[np.size(y)-1]] #current confirmed cases
pred = np.round(pol_reg.predict(poly_reg.fit_transform([[np.size(y)-1]])),0) #same day prediction
nxt_pred = np.round(pol_reg.predict(poly_reg.fit_transform([[np.size(y) + 0]])),0) #next day prediction
mortality_rate = np.round((dth/cnf)*100,2)
nxm = nxt_pred*mortality_rate/100 #Next day mortality prediction if mortality rate remains stable
print('---\n'+str(country + ' - Date: '+dates[np.size(dates)-1]))
print('\nCurrent cases: '+ str(cur) +  (' - Mortality rate: '+ str(mortality_rate) + '%') + '\nPredicted: ' + str(pred) + '\nDiff: ' + str(cur-pred)+'\nr-squared: '+ str(np.round(rsqrd,2)))
print('\nNext day prediction: '+str(nxt_pred) + ' - mortality prediction: '+ str(int(np.round(nxm,0))) + ' patients')
print('\nIncidents will double approximately in ' + str(double_days(x,y,xf,yf)) + ' days' + '\n---')

---
Portugal - Date: 3/31/20

Current cases: [7443.0] - Mortality rate: 2.15%
Predicted: [7200.]
Diff: [243.]
r-squared: 0.99

Next day prediction: [8099.] - mortality prediction: 174 patients
1000

Incidents will double approximately in 7 days
---
